Data Transformation Stage


In [1]:
import os

os.chdir("../")

In [2]:
%pwd

'd:\\cboobs\\textSummarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [ ]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
            )
        
        return data_transformation_config

In [10]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def examples2features(self, exampleBatch):
        inputEncodings = self.tokenizer(exampleBatch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            targetEncodings = self.tokenizer(exampleBatch['summary'], max_length=128, truncation=True)

        return{
            'input_ids': inputEncodings['input_ids'],
            'attention_mask': inputEncodings['attention_mask'],
            'labels': targetEncodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.examples2features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [11]:
try:
    config = ConfigManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-09-15 22:15:09,462: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-09-15 22:15:09,466: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-09-15 22:15:09,468: INFO: common: Created directory at: artifacts:]
[2025-09-15 22:15:09,469: INFO: common: Created directory at: artifacts/data_transformation:]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\cboobs\textSummarizer\textsummarizer\Lib\site-packages\transformers\tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 123136.08 examples/s]
